In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider


def lfm_chirp(t, t0, f0, k, T):
    """
    Generate an LFM chirp signal.

    Parameters:
    t : array-like
        Time samples.
    t0 : float
        Start time of the chirp.
    f0 : float
        Start frequency of the chirp (Hz).
    k : float
        Chirp rate (Hz/s).
    T : float
        Pulse duration (seconds).

    Returns:
    chirp_signal : array-like
        LFM chirp signal.
    """
    # Initialize the chirp signal with zeros
    t0 -= T/2
    chirp_signal = np.zeros_like(t, dtype=float)
    
    # Determine the times within the chirp duration
    mask = (t >= t0) & (t <= t0 + T)
    t_within = t[mask] - t0  # Time relative to the chirp start
    
    # Compute the instantaneous phase for the chirp
    phase = 2 * np.pi * (f0 * t_within + 0.5 * k * t_within**2)
    
    # Set the chirp signal for the valid times
    chirp_signal[mask] = np.cos(phase)
    
    return chirp_signal


def generate_overlap():
    x = np.linspace(-10, 10, 1000)
    waveform1 = lfm_chirp(x, 0, 1, 2, 2)
    # waveform1 = np.exp(-x**2 / 4)  # Gaussian function
    # returned1 = np.heaviside(x + 2, 1) - np.heaviside(x - 2, 1)  # Rectangular pulse
    returned1 = 0.5 * lfm_chirp(x, 0, 1, 2, 2)
    
    # Shift the second waveform
    offset = np.linspace(-500, 500, 1000)
    separation = np.arange(0, 8, 0.1)
    convolution = np.zeros((1000, len(separation)))
    for i, shift in enumerate(offset):
        for j, sep in enumerate(separation):
            returned1_shifted = np.roll(returned1, int(shift))
            waveform2 = lfm_chirp(x - sep, 0, 1, 2, 2)
            overlap = sum((waveform1 + waveform2) * returned1_shifted) / len(x)
            convolution[i, j] = overlap
    
    # Compute the overlap integral (convolution at this step)
    return convolution

convolution = generate_overlap()

def convolution_demo(slider=0, separation=0.1):
    """
    Interactive demonstration of convolution.
    offset: Amount to slide the second waveform relative to the first.
    """
    offset = slider - 500
    # Define the waveforms
    x = np.linspace(-10, 10, 1000)
    waveform1 = lfm_chirp(x, 0, 1, 2, 2)
    waveform2 = lfm_chirp(x - separation, 0, 1, 2, 2)
    # waveform1 = np.exp(-x**2 / 4)  # Gaussian function
    # waveform2 = np.heaviside(x + 2, 1) - np.heaviside(x - 2, 1)  # Rectangular pulse
    returned1 = 0.5 * lfm_chirp(x, 0, 1, 2, 2)
    
    # Shift the second waveform
    returned1_shifted = np.roll(returned1, offset)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Subplot 1: Waveforms
    plt.subplot(2, 1, 1)
    plt.plot(x, waveform1 + waveform2, label="Template")
    plt.plot(x, returned1_shifted, label="Return Signal")
    plt.title("Return Signal and Template")
    plt.xlabel("x")
    plt.ylabel("Amplitude")
    plt.ylim((-2, 2))
    plt.legend()
    plt.grid()
    
    # Subplot 2: Convolution value
    plt.subplot(2, 1, 2)
    plt.plot(x[0:slider], convolution[0:slider, int(separation * 10)])
    plt.title("Combined Output")
    plt.ylabel("Amplitude")
    plt.xlim((-10, 10))
    plt.ylim((-0.05, 0.05))
    plt.grid()
    
    plt.tight_layout()
    plt.show()

# Interactive slider for waveform offset
interact(
    convolution_demo,
    slider=IntSlider(value=0, min=50, max=950, step=1, description="Offset"),
    separation=IntSlider(value=1, min=0, max=7.9, step=0.1, description="Separation"),
)
